In [17]:
import numpy as np
import random
import scipy.sparse as sp
import torch
from time import time
import pandas as pd
import os
from datetime import datetime
import easydict
import tqdm
import scipy.sparse as sp
import torch.nn.functional as F
from torch import nn



In [154]:
for i in set([10, 20,30]) - set([10]):
    print(i)

20
30


In [149]:
x = torch.Tensor([[1,2,3,4,5,6], [1,2,3,4,5,7]])
print(x)
b = nn.Parameter(x)
print(x)
print(b[np.array([0,0,0,0,0])])


tensor([[1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 7.]])
tensor([[1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 7.]])
tensor([[1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.]], grad_fn=<IndexBackward0>)


In [2]:
# to utils
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed
    # unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True
set_seed(42)


In [3]:
args = easydict.EasyDict(
        {"data_dir": './data/', 
         "dataset": "ml-100k_ngcf", 
         "results_dir": '', 
         "n_epochs" : 40,# 400,
         "reg" : 1e-5,
         "lr" : 0.0001,
         "emb_dim" : 64,
         "layers" : '[64, 64]',
         "batch_size" : 1024,
         "node_dropout" : 0.1,
         "mess_dropout" : 0.1,
         "k" : 20, # 'k order of metric evaluation (e.g. NDCG@k)
         "eval_N" : 1,
         "save_results" : 1,
         "gpu_id" : 0 ,
         })

device = torch.device("cuda:" + str(args.gpu_id) if torch.cuda.is_available() else "cpu")


In [4]:
# read parsed arguments
data_dir = args.data_dir
dataset = args.dataset
batch_size = args.batch_size
layers = eval(args.layers)
emb_dim = args.emb_dim
lr = args.lr
reg = args.reg
mess_dropout = args.mess_dropout
node_dropout = args.node_dropout
k = args.k

In [46]:
# data_path는 사용자의 디렉토리에 맞게 설정해야 합니다.
data_path = '../data/train/train_ratings.csv'
df = pd.read_csv(data_path)

item_ids = df['item'].unique() # 아이템 고유 번호 리스트
user_ids = df['user'].unique() # 유저 고유 번호 리스트
num_item, num_user = len(item_ids), len(user_ids)
num_batch = num_user // batch_size

# user, item indexing
# item2idx = pd.Series(data=np.arange(len(item_ids))+1, index=item_ids) # item re-indexing (1~num_item) ; 아이템을 1부터 설정하는이유? 0을 아무것도 아닌 것으로 blank 하기 위해서
item2idx = pd.Series(data=np.arange(len(item_ids)), index=item_ids) # item re-indexing (0~num_item-1) ; 아이템을 1부터 설정하는이유? 0을 아무것도 아닌 것으로 blank 하기 위해서
user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids) # user re-indexing (0~num_user-1)

# dataframe indexing
df = pd.merge(df, pd.DataFrame({'item': item_ids, 'item_idx': item2idx[item_ids].values}), on='item', how='inner')
df = pd.merge(df, pd.DataFrame({'user': user_ids, 'user_idx': user2idx[user_ids].values}), on='user', how='inner')
df.sort_values(['user_idx', 'time'], inplace=True)
del df['item'], df['user'] 


In [47]:
print('num_item: ', num_item)
print('num_user: ', num_user)

num_item:  6807
num_user:  31360


In [48]:
# create interactions/ratings matrix 'R' # dok = dictionary of keys
R_train = sp.dok_matrix((num_user, num_item), dtype=np.float32) 
train_items, test_set = {}, {}

t1 = time() # timer
items = df.groupby("user_idx")["item_idx"].apply(list) # 유저 아이디 상관 없이, 순서대로 
for uid, item in tqdm.tqdm(enumerate(items)):
    for i in item:
        R_train[uid, i] = 1.
    train_items[uid] = item

print('Complete. Interaction matrices R_train and R_test created in', time() - t1, 'sec')


# if exist, get adjacency matrix
def get_adj_mat(self):
    try:
        t1 = time()
        adj_mat = sp.load_npz(self.path + '/s_adj_mat.npz')
        print('Loaded adjacency-matrix (shape:', adj_mat.shape,') in', time() - t1, 'sec.')

    except Exception:
        print('Creating adjacency-matrix...')
        adj_mat = self.create_adj_mat()
        sp.save_npz(self.path + '/s_adj_mat.npz', adj_mat)
    return adj_mat


31360it [00:55, 560.35it/s]

Complete. Interaction matrices R_train and R_test created in 57.48108887672424 sec


In [53]:
[0,1,2,3,4][0:]

[0, 1, 2, 3, 4]

In [54]:
# # if exist, get adjacency matrix
# def get_adj_mat(self):
#     try:
#         t1 = time()
#         adj_mat = sp.load_npz(self.path + '/s_adj_mat.npz')
#         print('Loaded adjacency-matrix (shape:', adj_mat.shape,') in', time() - t1, 'sec.')

#     except Exception:
#         print('Creating adjacency-matrix...')
#         adj_mat = self.create_adj_mat()
#         sp.save_npz(self.path + '/s_adj_mat.npz', adj_mat)
#     return adj_mat

# # create adjancency matrix
# def create_adj_mat(self):
#     t1 = time()
    
#     adj_mat = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
#     adj_mat = adj_mat.tolil()
#     R = self.R_train.tolil() # to list of lists

#     adj_mat[:self.n_users, self.n_users:] = R
#     adj_mat[self.n_users:, :self.n_users] = R.T
#     adj_mat = adj_mat.todok()
#     print('Complete. Adjacency-matrix created in', adj_mat.shape, time() - t1, 'sec.')

#     t2 = time()

#     # normalized adjacency matrix
#     def normalized_adj_single(adj):
        
#         ### 논문 수식 (8) ###
#         rowsum = np.array(adj.sum(1))                
#         d_inv = np.power(rowsum, -.5).flatten()     
#         d_inv[np.isinf(d_inv)] = 0.
#         d_mat_inv = sp.diags(d_inv)                 
#         norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv) 
        
#         return norm_adj.tocoo()

#     print('Transforming adjacency-matrix to Normalized-adjacency matrix...')
#     ngcf_adj_mat = normalized_adj_single(adj_mat)

#     print('Complete. Transformed adjacency-matrix to Normalized-adjacency matrix in', time() - t2, 'sec.')
#     return ngcf_adj_mat.tocsr()

def create_adj_mat():
    t1 = time()
    
    adj_mat = sp.dok_matrix((num_user + num_item, num_user + num_item), dtype=np.float32)
    adj_mat = adj_mat.tolil()
    R = R_train.tolil() # to list of lists

    adj_mat[:num_user, num_user:] = R
    adj_mat[num_user:, :num_user] = R.T
    adj_mat = adj_mat.todok()
    print('Complete. Adjacency-matrix created in', adj_mat.shape, time() - t1, 'sec.')

    t2 = time()

    # normalized adjacency matrix
    def normalized_adj_single(adj):
        
        ### 논문 수식 (8) ###
        rowsum = np.array(adj.sum(1))                
        d_inv = np.power(rowsum, -.5).flatten()     
        d_inv[np.isinf(d_inv)] = 0.
        d_mat_inv = sp.diags(d_inv)                 
        norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv) 
        
        return norm_adj.tocoo()

    print('Transforming adjacency-matrix to Normalized-adjacency matrix...')
    ngcf_adj_mat = normalized_adj_single(adj_mat)

    print('Complete. Transformed adjacency-matrix to Normalized-adjacency matrix in', time() - t2, 'sec.')
    return ngcf_adj_mat.tocsr()


print('Creating adjacency-matrix...')
adj_mat = create_adj_mat()


Creating adjacency-matrix...
(31360, 6807)
Complete. Adjacency-matrix created in (38167, 38167) 32.98339104652405 sec.
Transforming adjacency-matrix to Normalized-adjacency matrix...
Complete. Transformed adjacency-matrix to Normalized-adjacency matrix in 8.019442319869995 sec.


In [59]:
# create collections of N items that users never interacted with
neg_pools = {} # self.neg_pools = {}
def negative_pool():
    t1 = time()
    for u in train_items.keys():
        neg_items = list(set(range(num_item)) - set(train_items[u]))
        pools = [random.choice(neg_items) for _ in range(100)]
        neg_pools[u] = pools
    print('refresh negative pools', time() - t1)
negative_pool()


refresh negative pools 16.68658709526062


In [62]:
# sample data for mini-batches
def sample(self):
    # 배치 사이즈 = 유저의 수
    if self.batch_size <= self.n_users:
        users = random.sample(self.exist_users, self.batch_size)
    else:
        users = [random.choice(self.exist_users) for _ in range(self.batch_size)]

    # 유저 u에 대해서 positive item 실제로 본 영화도 다 넣는게 아니다. 매번 샘플링으로 결정한다. 
    def sample_pos_items_for_u(u, num):
        pos_items = self.train_items[u]
        n_pos_items = len(pos_items)
        pos_batch = []
        while True:
            if len(pos_batch) == num: break
            pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
            pos_i_id = pos_items[pos_id]

            if pos_i_id not in pos_batch:
                pos_batch.append(pos_i_id)
        return pos_batch

    # 유저 u가 보지 않았던 영화들을 num개 만큼 뽑는다.
    def sample_neg_items_for_u(u, num):
        neg_items = []
        while True:
            if len(neg_items) == num: break
            neg_id = np.random.randint(low=0, high=self.n_items -1 ,size=1)[0]
            if neg_id not in self.train_items[u] and neg_id not in neg_items:
                neg_items.append()
        return neg_items

    # 
    def sample_neg_items_for_u_from_pools(u, num):
        neg_items = list(set(self.neg_pools[u]) - set(self.train_items[u]))
        return random.sample(neg_items, num)

    # (batch size 만큼의) 유저에 대해서, 그 유저들의 pos item과 neg item 리스트를 만든다.    
    pos_items, neg_items = [], []
    for u in users:
        pos_items += sample_pos_items_for_u(u, 1)
        neg_items += sample_neg_items_for_u(u, 1)

    return users, pos_items, neg_items

def get_num_users_items(self):
    return self.n_users, self.n_items

def print_statistics(self):
    print('n_users=%d, n_items=%d' % (self.n_users, self.n_items))
    print('n_interactions=%d' % (self.n_train + self.n_test))
    print('n_train=%d, n_test=%d, sparsity=%.5f' % (self.n_train, self.n_test, (self.n_train + self.n_test)/(self.n_users * self.n_items)))


In [63]:
###############################################################################
# 주어진 결과와 정확히 비교하기 위한 random seed 고정
###############################################################################

seed = 0  # 바꾸지 마시오!
random.seed(seed)
np.random.seed(seed)

In [64]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device, use_cuda

(device(type='cuda'), True)

In [65]:
args = easydict.EasyDict(
        {"data_dir": './data/', 
         "dataset": "ml-100k_ngcf", 
         "results_dir": '', 
         "n_epochs" : 40,# 400,
         "reg" : 1e-5,
         "lr" : 0.0001,
         "emb_dim" : 64,
         "layers" : '[64, 64]',
         "batch_size" : 1024,
         "node_dropout" : 0.1,
         "mess_dropout" : 0.1,
         "k" : 20, # 'k order of metric evaluation (e.g. NDCG@k)
         "eval_N" : 1,
         "save_results" : 1,
         "gpu_id" : 0 ,
         })

device = torch.device("cuda:" + str(args.gpu_id) if torch.cuda.is_available() else "cpu")


In [125]:
import random as rd
class Data(object):
    def __init__(self, path, batch_size):
        self.path = '.'

        #get number of users and items
        self.n_users, self.n_items = 0, 0
        self.n_train, self.n_test = 0, 0
        self.neg_pools = {}

        self.exist_users = []

        self.batch_size = batch_size
        # data_path는 사용자의 디렉토리에 맞게 설정해야 합니다.
        data_path = '../data/train/train_ratings.csv'
        df = pd.read_csv(data_path)
        self.n_train = len(df)

        item_ids = df['item'].unique() # 아이템 고유 번호 리스트
        user_ids = df['user'].unique() # 유저 고유 번호 리스트
        num_item, num_user = len(item_ids), len(user_ids)
        num_batch = num_user // batch_size
        
        # user, item indexing
        # item2idx = pd.Series(data=np.arange(len(item_ids))+1, index=item_ids) # item re-indexing (1~num_item) ; 아이템을 1부터 설정하는이유? 0을 아무것도 아닌 것으로 blank 하기 위해서
        item2idx = pd.Series(data=np.arange(len(item_ids)), index=item_ids) # item re-indexing (0~num_item-1) ; 아이템을 1부터 설정하는이유? 0을 아무것도 아닌 것으로 blank 하기 위해서
        user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids) # user re-indexing (0~num_user-1)

        # dataframe indexing
        df = pd.merge(df, pd.DataFrame({'item': item_ids, 'item_idx': item2idx[item_ids].values}), on='item', how='inner')
        df = pd.merge(df, pd.DataFrame({'user': user_ids, 'user_idx': user2idx[user_ids].values}), on='user', how='inner')
        df.sort_values(['user_idx', 'time'], inplace=True)
        del df['item'], df['user']

        self.n_items = num_item
        self.n_users = num_user
        self.exist_users = list(df['user_idx'].unique())


        # create interactions/ratings matrix 'R' # dok = dictionary of keys
        print('Creating interaction matrices R_train and R_test...')
        t1 = time()
        self.R_train = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32) 
        self.R_test = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)

        self.train_items, self.test_set = {}, {}
        
        items = df.groupby("user_idx")["item_idx"].apply(list) # 유저 아이디 상관 없이, 순서대로 
        for uid, item in tqdm.tqdm(enumerate(items)):
            for i in item:
                self.R_train[uid, i] = 1.
            self.train_items[uid] = item

        print('Complete. Interaction matrices R_train and R_test created in', time() - t1, 'sec')        


    # if exist, get adjacency matrix
    def get_adj_mat(self):
        try:
            t1 = time()
            adj_mat = sp.load_npz(self.path + '/s_adj_mat.npz')
            print('Loaded adjacency-matrix (shape:', adj_mat.shape,') in', time() - t1, 'sec.')

        except Exception:
            print('Creating adjacency-matrix...')
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat.npz', adj_mat)
        return adj_mat
    
    # create adjancency matrix
    def create_adj_mat(self):
        t1 = time()
        
        adj_mat = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
        adj_mat = adj_mat.tolil()
        R = self.R_train.tolil() # to list of lists

        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T
        adj_mat = adj_mat.todok()
        print('Complete. Adjacency-matrix created in', adj_mat.shape, time() - t1, 'sec.')

        t2 = time()

        # normalized adjacency matrix
        def normalized_adj_single(adj):
            
            ### 논문 수식 (8) ###
            rowsum = np.array(adj.sum(1))                
            d_inv = np.power(rowsum, -.5).flatten()     
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)                 
            norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv) 
            
            return norm_adj.tocoo()

        print('Transforming adjacency-matrix to Normalized-adjacency matrix...')
        ngcf_adj_mat = normalized_adj_single(adj_mat)

        print('Complete. Transformed adjacency-matrix to Normalized-adjacency matrix in', time() - t2, 'sec.')
        return ngcf_adj_mat.tocsr()

    # create collections of N items that users never interacted with
    def negative_pool(self):
        t1 = time()
        for u in self.train_items.keys():
            neg_items = list(set(range(self.n_items)) - set(self.train_items[u]))
            pools = [rd.choice(neg_items) for _ in range(100)]
            self.neg_pools[u] = pools
        print('refresh negative pools', time() - t1)

    # sample data for mini-batches
    def sample(self):
        if self.batch_size <= self.n_users:
            users = rd.sample(self.exist_users, self.batch_size)
        else:
            users = [rd.choice(self.exist_users) for _ in range(self.batch_size)]

        def sample_pos_items_for_u(u, num):
            pos_items = self.train_items[u]
            n_pos_items = len(pos_items)
            pos_batch = []
            while True:
                if len(pos_batch) == num: break
                pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
                pos_i_id = pos_items[pos_id]

                if pos_i_id not in pos_batch:
                    pos_batch.append(pos_i_id)
            return pos_batch

        def sample_neg_items_for_u(u, num):
            neg_items = []
            while True:
                if len(neg_items) == num: break
                neg_id = np.random.randint(low=0, high=self.n_items,size=1)[0]
                if neg_id not in self.train_items[u] and neg_id not in neg_items:
                    neg_items.append(neg_id)
            return neg_items

        def sample_neg_items_for_u_from_pools(u, num):
            neg_items = list(set(self.neg_pools[u]) - set(self.train_items[u]))
            return rd.sample(neg_items, num)

        pos_items, neg_items = [], []
        for u in users:
            pos_items += sample_pos_items_for_u(u, 1)
            neg_items += sample_neg_items_for_u(u, 1)

        return users, pos_items, neg_items

    def get_num_users_items(self):
        return self.n_users, self.n_items

    def print_statistics(self):
        print('n_users=%d, n_items=%d' % (self.n_users, self.n_items))
        print('n_interactions=%d' % (self.n_train + self.n_test))
        print('n_train=%d, n_test=%d, sparsity=%.5f' % (self.n_train, self.n_test, (self.n_train + self.n_test)/(self.n_users * self.n_items)))


In [126]:
def early_stopping(log_value, best_value, stopping_step, flag_step, expected_order='asc'):
    """
    Check if early_stopping is needed
    Function copied from original code
    """
    assert expected_order in ['asc', 'des']
    if (expected_order == 'asc' and log_value >= best_value) or (expected_order == 'des' and log_value <= best_value):
        stopping_step = 0
        best_value = log_value
    else:
        stopping_step += 1

    if stopping_step >= flag_step:
        print("Early stopping at step: {} log:{}".format(flag_step, log_value))
        should_stop = True
    else:
        should_stop = False

    return best_value, stopping_step, should_stop


def train(model, data_generator, optimizer):
    """
    Train the model PyTorch style

    Arguments:
    ---------
    model: PyTorch model
    data_generator: Data object
    optimizer: PyTorch optimizer
    """
    model.train()
    n_batch = data_generator.n_train // data_generator.batch_size + 1
    running_loss=0
    for _ in range(n_batch):
        u, i, j = data_generator.sample()
        optimizer.zero_grad()
        loss = model(u,i,j)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss


def split_matrix(X, n_splits=100):
    """
    Split a matrix/Tensor into n_folds (for the user embeddings and the R matrices)

    Arguments:
    ---------
    X: matrix to be split
    n_folds: number of folds

    Returns:
    -------
    splits: split matrices
    """
    splits = []
    chunk_size = X.shape[0] // n_splits
    for i in range(n_splits):
        start = i * chunk_size
        end = X.shape[0] if i == n_splits - 1 else (i + 1) * chunk_size
        splits.append(X[start:end])
    return splits



In [127]:
def compute_ndcg_k(pred_items, test_items, test_indices, k):
    """
    Compute NDCG@k
    
    Arguments:
    ---------
    pred_items: binary tensor with 1s in those locations corresponding to the predicted item interactions
    test_items: binary tensor with 1s in locations corresponding to the real test interactions
    test_indices: tensor with the location of the top-k predicted items
    k: k'th-order 

    Returns:
    -------
    NDCG@k
    """
    r = (test_items * pred_items).gather(1, test_indices)
    f = torch.from_numpy(np.log2(np.arange(2, k+2))).float().to(device)
    
    dcg = (r[:, :k]/f).sum(1)                                               
    dcg_max = (torch.sort(r, dim=1, descending=True)[0][:, :k]/f).sum(1)   
    ndcg = dcg/dcg_max                                                     
    
    ndcg[torch.isnan(ndcg)] = 0
    return ndcg

def eval_model(u_emb, i_emb, Rtr, Rte, k):
    """
    Evaluate the model
    
    Arguments:
    ---------
    u_emb: User embeddings
    i_emb: Item embeddings
    Rtr: Sparse matrix with the training interactions
    Rte: Sparse matrix with the testing interactions
    k : kth-order for metrics
    
    Returns:
    --------
    result: Dictionary with lists correponding to the metrics at order k for k in Ks
    """
    # split matrices
    ue_splits = split_matrix(u_emb)
    tr_splits = split_matrix(Rtr)
    te_splits = split_matrix(Rte)

    recall_k, ndcg_k= [], []
    # compute results for split matrices
    for ue_f, tr_f, te_f in zip(ue_splits, tr_splits, te_splits):

        scores = torch.mm(ue_f, i_emb.t())

        test_items = torch.from_numpy(te_f.todense()).float().to(device)
        non_train_items = torch.from_numpy(1-(tr_f.todense())).float().to(device)
        scores = scores * non_train_items

        _, test_indices = torch.topk(scores, dim=1, k=k)
        

        pred_items = torch.zeros_like(scores).float()
        pred_items.scatter_(dim=1, index=test_indices, src=torch.ones_like(test_indices).float().to(device))

        topk_preds = torch.zeros_like(scores).float()
        topk_preds.scatter_(dim=1, index=test_indices[:, :k], src=torch.ones_like(test_indices).float())
        
        TP = (test_items * topk_preds).sum(1)                      
        rec = TP/test_items.sum(1)
   
        ndcg = compute_ndcg_k(pred_items, test_items, test_indices, k)

        recall_k.append(rec)
        ndcg_k.append(ndcg)

    return torch.cat(recall_k).mean(), torch.cat(ndcg_k).mean()


In [128]:
args = easydict.EasyDict(
        {"data_dir": './data/', 
         "dataset": "ml-100k_ngcf", 
         "results_dir": '', 
         "n_epochs" : 40,# 400,
         "reg" : 1e-5,
         "lr" : 0.0001,
         "emb_dim" : 64,
         "layers" : '[64, 64]',
         "batch_size" : 1024,
         "node_dropout" : 0.1,
         "mess_dropout" : 0.1,
         "k" : 20, # 'k order of metric evaluation (e.g. NDCG@k)
         "eval_N" : 1,
         "save_results" : 1,
         "gpu_id" : 0 ,
         })

device = torch.device("cuda:" + str(args.gpu_id) if torch.cuda.is_available() else "cpu")

In [129]:
# read parsed arguments
data_dir = args.data_dir
dataset = args.dataset
batch_size = args.batch_size
layers = eval(args.layers)
emb_dim = args.emb_dim
lr = args.lr
reg = args.reg
mess_dropout = args.mess_dropout
node_dropout = args.node_dropout
k = args.k

In [130]:
class NGCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim, layers, reg, node_dropout, mess_dropout, adj_mtx):
        super().__init__()

        # initialize Class attributes
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.l_matrix = adj_mtx
        self.l_plus_i_matrix = adj_mtx + sp.eye(adj_mtx.shape[0])
        self.reg = reg
        self.layers = layers
        self.n_layers = len(self.layers)
        self.node_dropout = node_dropout
        self.mess_dropout = mess_dropout

        # Initialize weights
        self.weight_dict = self._init_weights()
        print("Weights initialized.")

        # Create Matrix 'L+I', PyTorch sparse tensor of SP adjacency_mtx
        self.L_plus_I = self._convert_sp_mat_to_sp_tensor(self.l_plus_i_matrix)
        self.L = self._convert_sp_mat_to_sp_tensor(self.l_matrix)

    # initialize weights
    def _init_weights(self):
        print("Initializing weights...")
        weight_dict = nn.ParameterDict()

        initializer = torch.nn.init.xavier_uniform_
        
        weight_dict['user_embedding'] = nn.Parameter(initializer(torch.empty(self.n_users, self.emb_dim).to(device)))
        weight_dict['item_embedding'] = nn.Parameter(initializer(torch.empty(self.n_items, self.emb_dim).to(device)))

        weight_size_list = [self.emb_dim] + self.layers

        for k in range(self.n_layers):
            weight_dict['W_one_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_one_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
            
            weight_dict['W_two_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_two_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
           
        return weight_dict

    # convert sparse matrix into sparse PyTorch tensor
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res

    # apply node_dropout
    def _droupout_sparse(self, X):
        """
        Drop individual locations in X
        
        Arguments:
        ---------
        X = adjacency matrix (PyTorch sparse tensor)
        dropout = fraction of nodes to drop
        noise_shape = number of non non-zero entries of X
        """
        node_dropout_mask = ((self.node_dropout) + torch.rand(X._nnz())).floor().bool().to(device)
        i = X.coalesce().indices()
        v = X.coalesce()._values()
        i[:,node_dropout_mask] = 0
        v[node_dropout_mask] = 0
        X_dropout = torch.sparse.FloatTensor(i, v, X.shape).to(X.device)

        return  X_dropout.mul(1/(1-self.node_dropout))

    def forward(self, u, i, j):
        """
        Computes the forward pass
        
        Arguments:
        ---------
        u = user
        i = positive item (user interacted with item)
        j = negative item (user did not interact with item)
        """
        # apply drop-out mask
        L_plus_I_hat = self._droupout_sparse(self.L_plus_I) if self.node_dropout > 0 else self.L_plus_I
        L_hat = self._droupout_sparse(self.L) if self.node_dropout > 0 else self.L
        
        # 논문 수식 (1)
        # user, item 임베딩 벡터를 연결한다. 
        ego_embeddings = torch.cat([self.weight_dict['user_embedding'], self.weight_dict['item_embedding']], 0)

        final_embeddings = [ego_embeddings]

        # forward pass for 'n' propagation layers
        for k in range(self.n_layers):
            
            
            ########## Fill below ###########
            ### 논문 수식 (7) ###
            
            # (L+I)E
            side_L_plus_I_embeddings = torch.sparse.mm(L_plus_I_hat, ego_embeddings) #힌트 : use torch.sparse.mm 
            
            # (L+I)EW_1 + b_1
            simple_embeddings = torch.matmul(side_L_plus_I_embeddings, self.weight_dict['W_one_%d' %k]) + self.weight_dict['b_one_%d' % k] #힌트 : use torch.matmul, self.weight_dict['W_one_%d' % k], self.weight_dict['b_one_%d' % k]
            
            # LE
            side_L_embeddings = torch.sparse.mm(L_hat, ego_embeddings) #힌트 : use torch.sparse.mm                                
            
            # LEE
            interaction_embeddings = torch.mul(side_L_embeddings, ego_embeddings)            #힌트 : use torch.mul
                                             
            # LEEW_2 + b_2
            interaction_embeddings = torch.matmul(interaction_embeddings, self.weight_dict['W_two_%d' % k]) + self.weight_dict['b_two_%d' % k] #힌트 : use torch.matmul, self.weight_dict['W_two_%d' % k], self.weight_dict['b_two_%d' % k]

            # non-linear activation 
            ego_embeddings = simple_embeddings + interaction_embeddings #힌트: use simple_embeddings, interaction_embeddings
            
            ########## Fill above ###########
            
            
            # add message dropout
            mess_dropout_mask = nn.Dropout(self.mess_dropout)
            ego_embeddings = mess_dropout_mask(ego_embeddings)

            # Perform L2 normalization
            norm_embeddings = F.normalize(ego_embeddings, p=2, dim=1)
            
            ### 논문 수식 (9) ###
            final_embeddings.append(norm_embeddings)                                            

        
        final_embeddings = torch.cat(final_embeddings, 1)                           
    
        # back to user/item dimension
        u_final_embeddings, i_final_embeddings = final_embeddings.split([self.n_users, self.n_items], 0)

        self.u_final_embeddings = nn.Parameter(u_final_embeddings)
        self.i_final_embeddings = nn.Parameter(i_final_embeddings)
        
        u_emb = u_final_embeddings[u] # user embeddings
        p_emb = i_final_embeddings[i] # positive item embeddings
        n_emb = i_final_embeddings[j] # negative item embeddings
    
        
        ########## Fill below ###########
        ### 논문 수식 (10) ###
        
        y_ui =  sum(torch.mul(u_emb, p_emb))               # 힌트 : use torch.mul, sum() method                             
        y_uj =  sum(torch.mul(u_emb, n_emb))               # 힌트 : use torch.mul, sum() method 
        
        ########## Fill above ########### 
        
        
        
        ########## Fill below ########### 
        ### 논문 수식 (11) ###
        
        log_prob =  torch.log(torch.sigmoid(y_ui-y_uj)).mean()           # 힌트 : use torch.log, torch.sigmoid, mean() method
        bpr_loss = -log_prob        
        if self.reg > 0.:
            l2norm = (torch.sum(u_emb**2)/2. + torch.sum(p_emb**2)/2. + torch.sum(n_emb**2)/2.) / u_emb.shape[0]
            l2reg = self.reg*l2norm            # FILL HERE #
            bpr_loss += l2reg
        
        ########## Fill above ###########
        
        
        return bpr_loss


In [131]:
# read parsed arguments
data_dir = args.data_dir
dataset = args.dataset
batch_size = args.batch_size
layers = eval(args.layers)
emb_dim = args.emb_dim
lr = args.lr
reg = args.reg
mess_dropout = args.mess_dropout
node_dropout = args.node_dropout
k = args.k

In [132]:
# generate the Normalized-adjacency matrix
data_generator = Data(path=data_dir + dataset, batch_size=batch_size)
adj_mtx = data_generator.get_adj_mat()

# create model name and save
modelname =  "NGCF" + \
    "_bs_" + str(batch_size) + \
    "_nemb_" + str(emb_dim) + \
    "_layers_" + str(layers) + \
    "_nodedr_" + str(node_dropout) + \
    "_messdr_" + str(mess_dropout) + \
    "_reg_" + str(reg) + \
    "_lr_"  + str(lr)

# create NGCF model
model = NGCF(data_generator.n_users, 
             data_generator.n_items,
             emb_dim,
             layers,
             reg,
             node_dropout,
             mess_dropout,
             adj_mtx)

if use_cuda:
    model = model.cuda()

# current best metric
cur_best_metric = 0

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

# Set values for early stopping
cur_best_loss, stopping_step, should_stop = 1e3, 0, False
today = datetime.now()

print("Start at " + str(today))
print("Using " + str(device) + " for computations")
print("Params on CUDA: " + str(next(model.parameters()).is_cuda))

results = {"Epoch": [],
           "Loss": [],
           "Recall": [],
           "NDCG": [],
           "Training Time": []}


Creating interaction matrices R_train and R_test...


31360it [00:57, 545.92it/s]

Complete. Interaction matrices R_train and R_test created in 58.93110489845276 sec
Loaded adjacency-matrix (shape: (38167, 38167) ) in 0.0034456253051757812 sec.
Initializing weights...
Weights initialized.
Start at 2022-04-01 04:48:18.412060
Using cuda:0 for computations
Params on CUDA: True


In [134]:
for epoch in tqdm(range(args.n_epochs)):

    t1 = time()
    loss = train(model, data_generator, optimizer)
    training_time = time()-t1
    print("Epoch: {}, Training time: {:.2f}s, Loss: {:.4f}".
        format(epoch, training_time, loss))

    # print test evaluation metrics every N epochs (provided by args.eval_N)
    if epoch % args.eval_N  == (args.eval_N - 1):
        with torch.no_grad():
            t2 = time()
            recall, ndcg = eval_model(model.u_final_embeddings.detach(),
                                      model.i_final_embeddings.detach(),
                                      data_generator.R_train,
                                      data_generator.R_test,
                                      k)
        print(
            "Evaluate current model:\n",
            "Epoch: {}, Validation time: {:.2f}s".format(epoch, time()-t2),"\n",
            "Loss: {:.4f}:".format(loss), "\n",
            "Recall@{}: {:.4f}".format(k, recall), "\n",
            "NDCG@{}: {:.4f}".format(k, ndcg)
            )

        cur_best_metric, stopping_step, should_stop = \
        early_stopping(recall, cur_best_metric, stopping_step, flag_step=5)

        # save results in dict
        results['Epoch'].append(epoch)
        results['Loss'].append(loss)
        results['Recall'].append(recall.item())
        results['NDCG'].append(ndcg.item())
        results['Training Time'].append(training_time)
    else:
        # save results in dict
        results['Epoch'].append(epoch)
        results['Loss'].append(loss)
        results['Recall'].append(None)
        results['NDCG'].append(None)
        results['Training Time'].append(training_time)

    if should_stop == True: break


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 0, Training time: 477.43s, Loss: 710.7754


  2%|▎         | 1/40 [09:26<6:08:29, 566.91s/it]

Evaluate current model:
 Epoch: 0, Validation time: 89.48s 
 Loss: 710.7754: 
 Recall@20: nan 
 NDCG@20: 0.0000
Epoch: 1, Training time: 477.50s, Loss: 19.5138


  5%|▌         | 2/40 [18:54<5:59:23, 567.47s/it]

...

Evaluate current model:
 Epoch: 2, Validation time: 88.53s 
 Loss: 0.7031: 
 Recall@20: nan 
 NDCG@20: 0.0000
Epoch: 3, Training time: 476.87s, Loss: 0.2622


 10%|█         | 4/40 [37:47<5:40:01, 566.70s/it]

Evaluate current model:
 Epoch: 3, Validation time: 88.96s 
 Loss: 0.2622: 
 Recall@20: nan 
 NDCG@20: 0.0000
Epoch: 4, Training time: 473.29s, Loss: 0.2343


 10%|█         | 4/40 [47:10<7:04:31, 707.54s/it]

Evaluate current model:
 Epoch: 4, Validation time: 89.24s 
 Loss: 0.2343: 
 Recall@20: nan 
 NDCG@20: 0.0000
Early stopping at step: 5 log:nan
